**This part mainly looks at the balance of the data set and the effect of up and down sampling on the predictive performance of the model**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, cohen_kappa_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline


from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, cohen_kappa_score, classification_report
from sklearn.preprocessing import LabelBinarizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/fetal_health.csv')
data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [ ]:
# Check the distribution of the target variable 'sepsis'
fetal_distribution = data['fetal_health'].value_counts(normalize=True)
fetal_distribution

fetal_health
1.0    0.778457
2.0    0.138758
3.0    0.082785
Name: proportion, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Trying to reload the dataset and apply SMOTE
try:

    # Separating features and target
    X = data.drop('fetal_health', axis=1)
    y = data['fetal_health']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Applying SMOTE for upsampling
    smote = SMOTE(random_state=42)
    X_train_upsampled, y_train_upsampled = smote.fit_resample(X_train, y_train)

    # Checking the distribution after SMOTE
    upsample_distribution = y_train_upsampled.value_counts(normalize=True)
    upsample_result = upsample_distribution
except Exception as e:
    upsample_result = str(e)

upsample_result

# Check the new class distribution
print(y_train_upsampled.value_counts())

fetal_health
3.0    1323
1.0    1323
2.0    1323
Name: count, dtype: int64


In [ ]:
upsampled_data = pd.concat([X_train_upsampled, y_train_upsampled], axis=1)
upsampled_data

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,133.000000,0.000000,0.000000,0.012000,0.001000,0.000000,0.002000,60.000000,3.000000,0.0,...,58.000000,155.000000,4.000000,0.000000,125.000000,96.000000,105.000000,79.000000,0.0,3.0
1,146.000000,0.006000,0.000000,0.003000,0.000000,0.000000,0.000000,38.000000,1.000000,0.0,...,126.000000,175.000000,3.000000,0.000000,150.000000,152.000000,153.000000,5.000000,0.0,1.0
2,129.000000,0.000000,0.001000,0.007000,0.006000,0.000000,0.002000,67.000000,3.200000,0.0,...,66.000000,146.000000,7.000000,0.000000,105.000000,80.000000,107.000000,9.000000,0.0,3.0
3,134.000000,0.008000,0.001000,0.010000,0.006000,0.000000,0.000000,61.000000,1.100000,0.0,...,80.000000,189.000000,10.000000,0.000000,156.000000,144.000000,151.000000,61.000000,0.0,1.0
4,125.000000,0.000000,0.005000,0.001000,0.001000,0.000000,0.000000,70.000000,0.400000,29.0,...,52.000000,133.000000,5.000000,0.000000,125.000000,123.000000,125.000000,2.000000,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3964,129.000000,0.000000,0.001000,0.005932,0.008000,0.000000,0.002000,65.000000,2.806767,0.0,...,50.000000,151.000000,6.932332,2.000000,105.000000,85.864663,111.932332,12.932332,0.0,3.0
3965,131.168660,0.001000,0.000708,0.012292,0.009416,0.000000,0.001584,52.358558,3.397876,0.0,...,50.000000,207.786186,8.123505,0.876495,63.213814,94.258968,110.954846,239.774228,0.0,3.0
3966,110.000000,0.002913,0.001631,0.005195,0.008087,0.000000,0.002000,68.000000,3.108708,0.0,...,61.630615,189.738771,6.456462,0.543538,91.000000,82.108156,94.456462,39.912924,-1.0,3.0
3967,131.245176,0.000409,0.307785,0.002918,0.003000,0.000000,0.002082,36.288310,2.142792,0.0,...,55.389331,182.163451,9.509648,0.918275,76.144155,100.144155,102.653803,148.000000,-1.0,3.0


In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    scores = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred, average='macro'),
        'Precision': precision_score(y_test, y_pred, average='macro'),
        'F1 Score': f1_score(y_test, y_pred, average='macro'),
        'Kappa': cohen_kappa_score(y_test, y_pred),
        'AUC': roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovo', average='macro')
    }
    return scores


### original data GBoost matrix

In [ ]:
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
scores_without_smote = evaluate_model(model, X_test, y_test)
print("Performance without SMOTE:", scores_without_smote)


Performance without SMOTE: {'Accuracy': 0.9272300469483568, 'Recall': 0.8550477940818966, 'Precision': 0.880127723063821, 'F1 Score': 0.8637701463903719, 'Kappa': 0.7941740309533829, 'AUC': 0.9545725273977265}


### with smote GB(upsampling)

In [ ]:
smote_pipeline = make_pipeline(SMOTE(random_state=42), GradientBoostingClassifier(random_state=42))
smote_pipeline.fit(X_train, y_train)
scores_with_smote = evaluate_model(smote_pipeline, X_test, y_test)
print("Performance with SMOTE:", scores_with_smote)


Performance with SMOTE: {'Accuracy': 0.9295774647887324, 'Recall': 0.8839260188453572, 'Precision': 0.8805591521603606, 'F1 Score': 0.8822236586942469, 'Kappa': 0.8087370358730301, 'AUC': 0.9646117156276436}


In [ ]:
print("Performance Comparison:\n")
print("Metrics Without SMOTE:")
for key, value in scores_without_smote.items():
    print(f"{key}: {value:.4f}")

print("\nMetrics With SMOTE:")
for key, value in scores_with_smote.items():
    print(f"{key}: {value:.4f}")


Performance Comparison:

Metrics Without SMOTE:
Accuracy: 0.9272
Recall: 0.8550
Precision: 0.8801
F1 Score: 0.8638
Kappa: 0.7942
AUC: 0.9546

Metrics With SMOTE:
Accuracy: 0.9296
Recall: 0.8839
Precision: 0.8806
F1 Score: 0.8822
Kappa: 0.8087
AUC: 0.9646


### Random Forest

In [ ]:
def train_evaluate_model(X_train, y_train, X_test, y_test):
    # Initialize the model
    model = RandomForestClassifier(random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    kappa = cohen_kappa_score(y_test, y_pred)

    # AUC
    lb = LabelBinarizer()
    y_test_bin = lb.fit_transform(y_test)
    y_pred_bin = lb.transform(y_pred)
    auc = roc_auc_score(y_test_bin, y_pred_bin, average='macro', multi_class='ovr')

    return {
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1 Score': f1,
        'Kappa': kappa,
        'AUC': auc
    }

def apply_smote(X_train, y_train):
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    return X_train_smote, y_train_smote


In [ ]:
# Evaluate without SMOTE
results_without_smote = train_evaluate_model(X_train, y_train, X_test, y_test)

# Apply SMOTE
X_train_smote, y_train_smote = apply_smote(X_train, y_train)

# Evaluate with SMOTE
results_with_smote = train_evaluate_model(X_train_smote, y_train_smote, X_test, y_test)

# Print results
print("Original dataset:")
for key, value in results_without_smote.items():
    print(f"{key}: {value:.4f}")

print("\nResults with SMOTE:")
for key, value in results_with_smote.items():
    print(f"{key}: {value:.4f}")

print("\nResults with Downsampling:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


Original dataset:
Accuracy: 0.9249
Recall: 0.8285
Precision: 0.8770
F1 Score: 0.8499
Kappa: 0.7845
AUC: 0.9787

Results with SMOTE:
Accuracy: 0.9296
Recall: 0.8793
Precision: 0.8718
F1 Score: 0.8755
Kappa: 0.8088
AUC: 0.9817

Results with Downsampling:
Accuracy: 0.8850
Recall: 0.8827
Precision: 0.7904
F1 Score: 0.8283
Kappa: 0.7192
AUC: 0.9623


Model Results with Downsampling:

Accuracy: 0.8850

Recall: 0.8827

Precision: 0.7904

F1 Score: 0.8283

Kappa: 0.7192

AUC: 0.9623

## down sampling

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, cohen_kappa_score, classification_report
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
def apply_downsampling(X_train, y_train):
    # Define the downsampling method
    rus = RandomUnderSampler(random_state=42)

    # Apply downsampling
    X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

    return X_train_res, y_train_res

# Apply downsampling
X_train_res, y_train_res = apply_downsampling(X_train, y_train)


In [ ]:
def train_evaluate_model(X_train, y_train, X_test, y_test):
    # Initialize the model
    model = RandomForestClassifier(random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    kappa = cohen_kappa_score(y_test, y_pred)
    auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr', average='macro')

    return {
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1 Score': f1,
        'Kappa': kappa,
        'AUC': auc
    }


In [ ]:
# Evaluate the model
results = train_evaluate_model(X_train_res, y_train_res, X_test, y_test)

# Print results
print("Model Results with Downsampling:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


Model Results with Downsampling:
Accuracy: 0.8850
Recall: 0.8827
Precision: 0.7904
F1 Score: 0.8283
Kappa: 0.7192
AUC: 0.9623


### Original data with Logistic Regression, Decision Tree, KNN and SVC.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Logistic Regression
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)
scores_lr = evaluate_model(lr_model, X_test, y_test)
print("Performance without SMOTE (Logistic Regression):", scores_lr)

# Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
scores_dt = evaluate_model(dt_model, X_test, y_test)
print("Performance without SMOTE (Decision Tree):", scores_dt)

# K-Nearest Neighbors
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
scores_knn = evaluate_model(knn_model, X_test, y_test)
print("Performance without SMOTE (K-Nearest Neighbors):", scores_knn)

# Support Vector Classifier
svc_model = SVC(probability=True, random_state=42)
svc_model.fit(X_train, y_train)
scores_svc = evaluate_model(svc_model, X_test, y_test)
print("Performance without SMOTE (SVC):", scores_svc)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Performance without SMOTE (Logistic Regression): {'Accuracy': 0.8474178403755869, 'Recall': 0.6675870552427629, 'Precision': 0.7419419125093927, 'F1 Score': 0.6966582001095672, 'Kappa': 0.5332726537216829, 'AUC': 0.8692073378258799}
Performance without SMOTE (Decision Tree): {'Accuracy': 0.9014084507042254, 'Recall': 0.8091528340966773, 'Precision': 0.8247315362699977, 'F1 Score': 0.8159515832839358, 'Kappa': 0.7244501940491591, 'AUC': 0.856864625572508}
Performance without SMOTE (K-Nearest Neighbors): {'Accuracy': 0.8943661971830986, 'Recall': 0.749604714256542, 'Precision': 0.8299732341519085, 'F1 Score': 0.7844259290883834, 'Kappa': 0.6920927094877849, 'AUC': 0.9042033752443187}
Performance without SMOTE (SVC): {'Accuracy': 0.8544600938967136, 'Recall': 0.63883475791787, 'Precision': 0.7578114417914167, 'F1 Score': 0.6837606837606837, 'Kappa': 0.5554208958238651, 'AUC': 0.8897697326837616}
